In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv')

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [6]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [12]:
df.isnull().sum()

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

In [10]:
df1= df.drop(columns=["date", "lights"])

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler =MinMaxScaler()
normalised_df1 = pd.DataFrame(scaler.fit_transform(df1),columns=df1.columns)
#seperating the predictors and target variables
features_df =normalised_df1.drop(columns=['Appliances'])
appliances_target =normalised_df1['Appliances']

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features_df, appliances_target, test_size=0.3, random_state=42)


In [13]:
from sklearn.linear_model import LinearRegression

linear_model= LinearRegression()
#fit the model to the training dataset
linear_model.fit(x_train, y_train)
#obtain predictions
predicted_values = linear_model.predict(x_test)

In [14]:
#r-squared
from sklearn.metrics import r2_score
r2_score= r2_score(y_test, predicted_values)
round(r2_score, 2)

0.15

In [15]:
#MEA
from sklearn.metrics import mean_absolute_error
MAE=mean_absolute_error(y_test, predicted_values)
round(MAE,2)

0.05

In [16]:
#residual sum of squares(rss)
rss=np.sum(np.square(y_test - predicted_values))
round(rss, 2)

45.35

In [17]:
#root mean square error(rmse)
from sklearn.metrics import mean_squared_error
rmse= np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.088

In [18]:
#ridge regression
from sklearn.linear_model import Ridge
ridge_reg= Ridge(alpha=0.5)
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.5)

In [19]:
from sklearn.linear_model import Lasso
lasso_reg =Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001)

In [22]:
def get_weights_df1(model,feat, col_name):
    #this function returns weight of every feature
    weights= pd.Series(model.coef_, feat.columns).sort_values()
    weights_df1= pd.DataFrame(weights).reset_index()
    weights_df1.columns= ['Features', col_name]
    weights_df1[col_name].round(3)
    return weights_df1

linear_model_weights = get_weights_df1(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df1= get_weights_df1(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df1= get_weights_df1(lasso_reg, x_train, 'Lasso_Weight')

final_weights= pd.merge(linear_model_weights, ridge_weights_df1, on ='Features')
final_weights= pd.merge(final_weights, lasso_weights_df1, on= "Features")
print(final_weights)

       Features  Linear_Model_Weight  Ridge_Weight  Lasso_Weight
0           rv2        -7.160207e+08      0.000743     -0.000000
1          RH_2        -4.566990e-01     -0.401134     -0.000000
2         T_out        -3.218593e-01     -0.250765      0.000000
3            T2        -2.361788e-01     -0.193880      0.000000
4            T9        -1.899408e-01     -0.188584     -0.000000
5          RH_8        -1.575958e-01     -0.156596     -0.000110
6        RH_out        -7.767021e-02     -0.050541     -0.049557
7          RH_7        -4.461344e-02     -0.046291     -0.000000
8          RH_9        -3.980033e-02     -0.041701     -0.000000
9            T5        -1.565703e-02     -0.020727     -0.000000
10           T1        -3.281051e-03     -0.021549      0.000000
11  Press_mm_hg         6.839462e-03      0.006516     -0.000000
12           T7         1.031869e-02      0.010021     -0.000000
13   Visibility         1.230656e-02      0.012032      0.000000
14         RH_5         1

In [23]:
from sklearn.linear_model import Ridge
ridge_reg= Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.4)

In [24]:
pred1= ridge_reg.predict(x_test)

In [25]:
rmse= np.sqrt(mean_squared_error(y_test, pred1))
round(rmse, 3)

0.088

In [26]:
from sklearn.linear_model import Lasso
lasso_reg =Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001)

In [27]:
pred2=lasso_reg.predict(x_test)

In [28]:
rmse= np.sqrt(mean_squared_error(y_test, pred2))
round(rmse, 3)

0.094